In [1]:
import numpy as np, os,copy
import hytools as ht
import glob

os.chdir('/Volumes/Extreme_SSD/mangroves/imagery') 

input_files = []
for input in glob.glob('*ort'):
    input_files.append(input)
    
print(input_files)

['prm20140119t182713_rfl_ort', 'prm20140119t182133_rfl_ort', 'prm20140119t184451_rfl_ort', 'prm20140119t185056_rfl_ort', 'prm20140119t190307_rfl_ort', 'prm20140119t185651_rfl_ort', 'prm20140119t183909_rfl_ort', 'prm20140119t183310_rfl_ort', 'prm20140119t191543_rfl_ort', 'prm20140119t190852_rfl_ort']


In [3]:
#%%capture  #prevent obnoxious printout
for i in input_files:
    hyObj = ht.openENVI(i)
    hyObj.load_data()
    hyObj.create_bad_bands([[299,450],[1050,1110]])
    
    # Create output file
    output_file = i + "_scaled"
    header_dict = copy.copy(hyObj.header_dict)
    header_dict['interleave'] = 'bsq' 
    
    #This line changes the datatype to uint16
# see https://www.harrisgeospatial.com/docs/ENVIHeaderFiles.html
    header_dict['data type'] =12

# Here we pick a number to scale the data by since the data are between 0 and 1
# the max value will be 10,000 which is less than 2^16 = 65536
    scaler = 10000

#Change no data values from -50 to some unrealistic number
    header_dict['data ignore value'] = 2**16-1

#Here we use the bad bands mask to  update the wavelegnths
    header_dict['wavelength'] =hyObj.wavelengths[hyObj.bad_bands]

# We also need to update the number of bands
    header_dict['bands'] = len(hyObj.wavelengths[hyObj.bad_bands])

    writer = ht.file_io.writeENVI(output_file,header_dict)
    
    iterator = hyObj.iterate(by = 'band')

    new_band = 0
    while not iterator.complete:
        band = iterator.read_next()
    #Only process good bands
        if hyObj.bad_bands[iterator.current_band]:
            print(hyObj.wavelengths[iterator.current_band])
        #scale the band
            scaled_band = band*scaler
        #Change negatives to 0
            scaled_band[scaled_band<0] = 0
        #Change no data value
            scaled_band[band==hyObj.no_data ] = header_dict['data ignore value']

            writer.write_band(scaled_band,new_band)
            new_band +=1
        
    writer.close()
    
    

KeyboardInterrupt: 

In [ ]:
#####Setup#####

#load modules
#import hytools as ht
#from spectral import *
#import spectral.io.envi as envi
#import numpy as np
#import matplotlib as plt
#%matplotlib inlxine
#import mpld3
#mpld3.enable_notebook()

#Read in imagery data
#first test: clean two flightlines and try to mosaic them
#img1 = envi.open('/Volumes/Extreme_SSD/mangroves/imagery/20140119/prm20140119t182133_rfl_ort.hdr',
#                '/Volumes/Extreme_SSD/mangroves/imagery/20140119/prm20140119t182133_rfl_ort.dat')

#img2 = envi.open('/Volumes/Extreme_SSD/mangroves/imagery/20140119/prm20140119t182713_rfl_ort.hdr',
#                '/Volumes/Extreme_SSD/mangroves/imagery/20140119/prm20140119t182713_rfl_ort')

In [4]:
#view = imshow(hyObj.data.clip[:,:,[120,88,62]], stretch=0.08,title="RGB Image of 2014 PRISM subset" )

In [36]:
#first step is to remove noisy bands
#first pass remove everything below 450 and above 1050 nm

#get band positions
#for index, value in enumerate(img1.bands.centers, 0):
#     print("{}. {}".format(index, value))
        
#0-54 need to be removed; 265-284 need to be axed
#img_clean1 = img1[:,:,55:200]
#img_clean2 = img2[:,:,55:264]
